# Sentiment Analysis

### Setup

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import nltk.sentiment.vader as en
import vaderSentiment_fr.vaderSentiment as fr

X:\Dev\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Importing the (clean) data and filling empty values
df = pd.read_excel("../data/enquete_clean.xlsx")
df = df.fillna(' ')
df

,ID,"Si vous effectuez d'autres tâches, veuillez l'indiquer ci-dessous",Quelles sont vos aspirations futures ?,"Y a-t-il un facteur supplémentaire qui vous semble utile de mentionner ? Si oui, lequel ?",A quel niveau le placeriez-vous dans le ranking ci-dessus ? (1 = le PLUS important - 10 = le MOINS important),"Si oui ou non, expliquez pourquoi ...","Si autre, à préciser ... :1","Si autre, à préciser ... :2","Si autre, à préciser ... :3","Si autre, à préciser ... :4","Si autre, à préciser ... :5","Si autre, à préciser ... :6","Si autre, à préciser ... :7","Si autre, à préciser ... :8",Commentaire libre pour vous exprimer sur votre niveau de stress :,"Selon vous, quels sont les 3 aspects les plus intéressants chez John Cockerill ?","Selon vous, quels sont les 3 aspects les plus pénibles chez John Cockerill ?","Selon vous, quelle est la première action à prendre pour valoriser le métier ?","Si vous le souhaitez, n'hésitez pas à ajouter un commentaire. \n\nNous vous remercions d'avoir pris le temps de répondre à cette enquête",lang
0,1,,Une croissance sur des projets techniques plus...,,,J’ai l’impression que l’entreprise accorde bea...,,,,,,,,,,La culture du groupe \nLa structuration et l’h...,L’absence de vision à long terme \nLe manque d...,Donner davantage d’autonomie et de pouvoir au ...,,fr
1,2,,continuer à gérer des équipes de projets,"arrêter cette ambiguïté du tandem PM et TPM, i...",5,ce serait une redite,,,,,,,,,,"diversité des business, excepté la défense.\nl...",entreprise qui donne une image moderne mais qu...,"remettre le PM au centre du projet, c'est le p...",bonne chance,fr
2,3,,j'ai répondu au questionnaire comme si j étais...,Difficulté d'avoir accès à des ressources de q...,1,Ancrage des formations suivies en interne\nVal...,,,,,,,,,,"Variété des projets, niveau de responsabilité ...","combat permanent pour les ressources, portefeu...",réduire la charge horaire \ntrouver des moyens...,,fr
3,4,"administratives : gestion des documents, title...",Avoir une équipe à gérer de chargés d'affaires...,reconnaissance salariale,3,déjà certifié,,,,,,,,,,environnement international\nnouveaux produits...,la charge administrative et trop d'outils diff...,perspective d'avenir : A-t-on vraiment envie ...,,fr
4,5,,Rester Project manager,,,Pas le temps personnel a accorder au vu de la ...,,,,,,,,,,Groupe à taille humaine\nAutonomie\nDiversité,Lourdeur administrative\nManque de support inf...,Reconnaissance hiéerarchique,,fr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,99,Interface Management between the entities,"Progressively lead the Company, Buniness line ...",Better Project Management organisation (Staffing),3,Yes surely.\nHowever time and the heavy worklo...,Clear roles and responsibility in case of shar...,,,Engineering and technology - Emphasize focus o...,,,Project Monitoring and Control,,Lack of Consistancy in terms of support (Suppo...,Flexibility \nTrust \nOpportunity,Inconsistancy\nDecision making\nLoosing Techni...,Aggressive in the market and Pragmatism in exe...,"Change is Inevitable, but constantantly changi...",en
99,100,,"Team management, with technical aspects.",,,It's always interesting to have training / cer...,,,,,,,,-,,Freedom given to the project manager\nNew tech...,Complexity of promocore\nDificulty to have a c...,Recognize at is just value the work and give f...,,en
100,101,,"To be better Communicator, adaptable ,agile",,4,I am PMP Certified,,,,,,,,,,1. The way John Cockerill's presence in differ...,"1. Strong Functional matrix, in spite of being...",1. Professional Ethics\n2. Proper and proactiv...,Thank you for giving this opportunity to expre...,en
101,102,,To lead team on group level for internationa...,,1,PMP certification will help us to apply worl...,,,,,,,,,Improper Work load management of individual &...,"1.Tools like PROMOCORE, SAP & MSP help a lot ...",1. Technical information sharing within differ...,good training & global exposure to the team,1. I

In [3]:
# Putting the columns in a value
cols = df.columns.tolist()
cols

['ID',
 "Si vous effectuez d'autres tâches, veuillez l'indiquer ci-dessous",
 'Quelles sont vos aspirations futures ?',
 'Y a-t-il un facteur supplémentaire qui vous semble utile de mentionner ? Si oui, lequel ?',
 'A quel niveau le placeriez-vous dans le ranking ci-dessus ? (1 = le PLUS important - 10 = le MOINS important)',
 'Si oui ou non, expliquez pourquoi ...',
 'Si autre, à préciser ... :1',
 'Si autre, à préciser ... :2',
 'Si autre, à préciser ... :3',
 'Si autre, à préciser ... :4',
 'Si autre, à préciser ... :5',
 'Si autre, à préciser ... :6',
 'Si autre, à préciser ... :7',
 'Si autre, à préciser ... :8',
 'Commentaire libre pour vous exprimer sur votre niveau de stress :',
 'Selon vous, quels sont les 3 aspects les plus intéressants chez John Cockerill ?',
 'Selon vous, quels sont les 3 aspects les plus pénibles chez John Cockerill ?',
 'Selon vous, quelle est la première action à prendre pour valoriser le métier ?',
 "Si vous le souhaitez, n'hésitez pas à ajouter un comm

In [4]:
# Removing columns that are not interesting for this Sentiment Analysis
to_remove = ['ID',
            'lang',
            "Si vous effectuez d'autres tâches, veuillez l'indiquer ci-dessous",
            'A quel niveau le placeriez-vous dans le ranking ci-dessus ? (1 = le PLUS important - 10 = le MOINS important)',
           ]
for col in to_remove:
    cols.remove(col)

In [5]:
cols

['Quelles sont vos aspirations futures ?',
 'Y a-t-il un facteur supplémentaire qui vous semble utile de mentionner ? Si oui, lequel ?',
 'Si oui ou non, expliquez pourquoi ...',
 'Si autre, à préciser ... :1',
 'Si autre, à préciser ... :2',
 'Si autre, à préciser ... :3',
 'Si autre, à préciser ... :4',
 'Si autre, à préciser ... :5',
 'Si autre, à préciser ... :6',
 'Si autre, à préciser ... :7',
 'Si autre, à préciser ... :8',
 'Commentaire libre pour vous exprimer sur votre niveau de stress :',
 'Selon vous, quels sont les 3 aspects les plus intéressants chez John Cockerill ?',
 'Selon vous, quels sont les 3 aspects les plus pénibles chez John Cockerill ?',
 'Selon vous, quelle est la première action à prendre pour valoriser le métier ?',
 "Si vous le souhaitez, n'hésitez pas à ajouter un commentaire. \n\nNous vous remercions d'avoir pris le temps de répondre à cette enquête"]

In [6]:
# Creating a dataframe for each question
qsts = []
for col in cols:
    x = df[['ID', col, 'lang']]
    qsts.append(x)
qsts[0]

,ID,Quelles sont vos aspirations futures ?,lang
0,1,Une croissance sur des projets techniques plus...,fr
1,2,continuer à gérer des équipes de projets,fr
2,3,j'ai répondu au questionnaire comme si j étais...,fr
3,4,Avoir une équipe à gérer de chargés d'affaires...,fr
4,5,Rester Project manager,fr
...,...,...,...
98,99,"Progressively lead the Company, Buniness line ...",en
99,100,"Team management, with technical aspects.",en
100,101,"To be better Communicator, adaptable ,agile",en
101,102,To lead team on group level for internationa...,en


### Sentiment Analysis

In [7]:
# Loading VADER Sentiment Analysis models for English and French
SIA_en = en.SentimentIntensityAnalyzer()
SIA_fr = fr.SentimentIntensityAnalyzer()


In [8]:
# Function to calculate the sentiment for each answer
def calculate_sentiment(df):
    df_en = df[df['lang']=='en']
    df_en['scores'] = df_en.iloc[:, 1].apply(lambda text: SIA_en.polarity_scores(text)) 
    
    df_fr = df[df['lang']=='fr']
    df_fr['scores'] = df_fr.iloc[:, 1].apply(lambda text: SIA_fr.polarity_scores(text)) 
    
    df = pd.concat([df_fr, df_en])
    
    df.drop(['ID', 'lang'], axis=1, inplace=True)
    
    df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
    df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
    
    return df



In [9]:
# Applying the sentiment models to each question
qsts_and_sentmnt = []
for qst in qsts:
    x = calculate_sentiment(qst)
    qsts_and_sentmnt.append(x)
    
qsts_and_sentmnt[0]

<ipython-input-8-9e92a5f739a5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en['scores'] = df_en.iloc[:, 1].apply(lambda text: SIA_en.polarity_scores(text))
<ipython-input-8-9e92a5f739a5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fr['scores'] = df_fr.iloc[:, 1].apply(lambda text: SIA_fr.polarity_scores(text))


,Quelles sont vos aspirations futures ?,scores,compound,comp_score
0,Une croissance sur des projets techniques plus...,"{'neg': 0.0, 'neu': 0.6, 'pos': 0.4, 'compound...",0.4588,pos
1,continuer à gérer des équipes de projets,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
2,j'ai répondu au questionnaire comme si j étais...,"{'neg': 0.053, 'neu': 0.947, 'pos': 0.0, 'comp...",-0.4295,neg
3,Avoir une équipe à gérer de chargés d'affaires...,"{'neg': 0.103, 'neu': 0.686, 'pos': 0.211, 'co...",0.3595,pos
4,Rester Project manager,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
...,...,...,...,...
98,"Progressively lead the Company, Buniness line ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
99,"Team management, with technical aspects.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
100,"To be better Communicator, adaptable ,agile","{'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'comp...",0.4404,pos
101,To lead team on group level for internationa...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos


In [10]:
# Putting all responses and their sentiment in one dataframe
final = pd.concat(qsts_and_sentmnt, axis=1)
final

,Quelles sont vos aspirations futures ?,scores,compound,comp_score,"Y a-t-il un facteur supplémentaire qui vous semble utile de mentionner ? Si oui, lequel ?",scores,compound,comp_score,"Si oui ou non, expliquez pourquoi ...",scores,...,compound,comp_score,"Selon vous, quelle est la première action à prendre pour valoriser le métier ?",scores,compound,comp_score,"Si vous le souhaitez, n'hésitez pas à ajouter un commentaire. \n\nNous vous remercions d'avoir pris le temps de répondre à cette enquête",scores,compound,comp_score
0,Une croissance sur des projets techniques plus...,"{'neg': 0.0, 'neu': 0.6, 'pos': 0.4, 'compound...",0.4588,pos,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,J’ai l’impression que l’entreprise accorde bea...,"{'neg': 0.097, 'neu': 0.83, 'pos': 0.072, 'com...",...,0.3400,pos,Donner davantage d’autonomie et de pouvoir au ...,"{'neg': 0.074, 'neu': 0.877, 'pos': 0.049, 'co...",-0.1779,neg,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
1,continuer à gérer des équipes de projets,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,"arrêter cette ambiguïté du tandem PM et TPM, i...","{'neg': 0.14, 'neu': 0.86, 'pos': 0.0, 'compou...",-0.4545,neg,ce serait une redite,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",...,-0.2584,neg,"remettre le PM au centre du projet, c'est le p...","{'neg': 0.0, 'neu': 0.911, 'pos': 0.089, 'comp...",0.5983,pos,bonne chance,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4404,pos
2,j'ai répondu au questionnaire comme si j étais...,"{'neg': 0.053, 'neu': 0.947, 'pos': 0.0, 'comp...",-0.4295,neg,Difficulté d'avoir accès à des ressources de q...,"{'neg': 0.138, 'neu': 0.712, 'pos': 0.15, 'com...",-0.2458,neg,Ancrage des formations suivies en interne\nVal...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",...,-0.5859,neg,réduire la charge horaire \ntrouver des moyens...,"{'neg': 0.108, 'neu': 0.719, 'pos': 0.174, 'co...",0.2732,pos,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
3,Avoir une équipe à gérer de chargés d'affaires...,"{'neg': 0.103, 'neu': 0.686, 'pos': 0.211, 'co...",0.3595,pos,reconnaissance salariale,"{'neg': 0.0, 'neu': 0.345, 'pos': 0.655, 'comp...",0.2263,pos,déjà certifié,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",...,-0.4019,neg,perspective d'avenir : A-t-on vraiment envie ...,"{'neg': 0.092, 'neu': 0.733, 'pos': 0.175, 'co...",0.6214,pos,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
4,Rester Project manager,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,Pas le temps personnel a accorder au vu de la ...,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",...,-0.0772,neg,Reconnaissance hiéerarchique,"{'neg': 0.0, 'neu': 0.345, 'pos': 0.655, 'comp...",0.2263,pos,,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Progressively lead the Company, Buniness line ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,Better Project Management organisation (Staffing),"{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compou...",0.4404,pos,Yes surely.\nHowever time and the heavy worklo...,"{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'comp...",...,0.0000,pos,Aggressive in the market and Pragmatism in exe...,"{'neg': 0.096, 'neu': 0.904, 'pos': 0.0, 'comp...",-0.1531,neg,"Change is Inevitable, but constantantly changi...","{'neg': 0.0, 'neu': 0.693, 'pos': 0.307, 'comp...",0.7964,pos
99,"Team management, with technical aspects.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0.0000,pos,It's always interesting to have training / cer...,"{'neg': 0.0, 'neu': 0.616, 'pos': 0.384, 'comp...",...,0.4588,pos,Recognize at is just value the work and give f...,"{'neg': 0.0, 'neu': 0.662, 'pos': 0.338, 'comp...",0.5574,pos,,"{'neg': 0.0

In [11]:
#final.to_excel("../data/sentiment_analysis.xlsx")

In [12]:
# Creating a simple file with just the compound score (so we can colot it in excel)
final_simple = final.drop(['scores', 'comp_score'], axis=1)
final_simple

,Quelles sont vos aspirations futures ?,compound,"Y a-t-il un facteur supplémentaire qui vous semble utile de mentionner ? Si oui, lequel ?",compound,"Si oui ou non, expliquez pourquoi ...",compound,"Si autre, à préciser ... :1",compound,"Si autre, à préciser ... :2",compound,...,Commentaire libre pour vous exprimer sur votre niveau de stress :,compound,"Selon vous, quels sont les 3 aspects les plus intéressants chez John Cockerill ?",compound,"Selon vous, quels sont les 3 aspects les plus pénibles chez John Cockerill ?",compound,"Selon vous, quelle est la première action à prendre pour valoriser le métier ?",compound,"Si vous le souhaitez, n'hésitez pas à ajouter un commentaire. \n\nNous vous remercions d'avoir pris le temps de répondre à cette enquête",compound
0,Une croissance sur des projets techniques plus...,0.4588,,0.0000,J’ai l’impression que l’entreprise accorde bea...,-0.1779,,0.0000,,0.0,...,,0.0000,La culture du groupe \nLa structuration et l’h...,0.0000,L’absence de vision à long terme \nLe manque d...,0.3400,Donner davantage d’autonomie et de pouvoir au ...,-0.1779,,0.0000
1,continuer à gérer des équipes de projets,0.0000,"arrêter cette ambiguïté du tandem PM et TPM, i...",-0.4545,ce serait une redite,0.0000,,0.0000,,0.0,...,,0.0000,"diversité des business, excepté la défense.\nl...",0.1027,entreprise qui donne une image moderne mais qu...,-0.2584,"remettre le PM au centre du projet, c'est le p...",0.5983,bonne chance,0.4404
2,j'ai répondu au questionnaire comme si j étais...,-0.4295,Difficulté d'avoir accès à des ressources de q...,-0.2458,Ancrage des formations suivies en interne\nVal...,0.0000,,0.0000,,0.0,...,,0.0000,"Variété des projets, niveau de responsabilité ...",0.0000,"combat permanent pour les ressources, portefeu...",-0.5859,réduire la charge horaire \ntrouver des moyens...,0.2732,,0.0000
3,Avoir une équipe à gérer de chargés d'affaires...,0.3595,reconnaissance salariale,0.2263,déjà certifié,0.0000,,0.0000,,0.0,...,,0.0000,environnement international\nnouveaux produits...,-0.0516,la charge administrative et trop d'outils diff...,-0.4019,perspective d'avenir : A-t-on vraiment envie ...,0.6214,,0.0000
4,Rester Project manager,0.0000,,0.0000,Pas le temps personnel a accorder au vu de la ...,0.3612,,0.0000,,0.0,...,,0.0000,Groupe à taille humaine\nAutonomie\nDiversité,0.0000,Lourdeur administrative\nManque de support inf...,-0.0772,Reconnaissance hiéerarchique,0.2263,,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Progressively lead the Company, Buniness line ...",0.0000,Better Project Management organisation (Staffing),0.4404,Yes surely.\nHowever time and the heavy worklo...,0.6808,Clear roles and responsibility in case of shar...,0.8625,,0.0,...,Lack of Consistancy in terms of support (Suppo...,0.4504,Flexibility \nTrust \nOpportunity,0.8176,Inconsistancy\nDecision making\nLoosing Techni...,0.0000,Aggressive in the market and Pragmatism in exe...,-0.1531,"Change is Inevitable, but constantantly changi...",0.7964
99,"Team management, with technical aspects.",0.0000,,0.0000,It's always interesting to have training / cer...,0.6808,,0.0000,,0.0,...,,0.0000,Freedom given to the project manager\nNew tech...,0.6369,Complexity of promocore\nDificulty to have a c...,0.4588,Recognize at is just value the work and give f...,0.5574,,0.0000
100,"To be better Communicator, adaptable ,agile",0.4404,,0.0000,I am PMP Certified,0.0000,,0.0000,,0.0,...,,0.0000,1. The way John Cockerill's presence in differ...,0.7964,"1. Strong Functional matrix, in spite of being...",-0.3400,1. Professional Ethics\n2. Proper and proactiv...,0.5859,Thank you for giving this opportunity to expre...,0.9072
101,To lead team on group level for internationa...,0.0000,,0.0000,PMP certification will help us to apply worl...,0.8625,,0.0000,,0.0,...,Improper Work load management of individual &...,0.6327,"1.Tools like PROMOCORE, SAP & MSP help a lot ...",0.7906,1. Technical information sharing wi

In [13]:
# final_simple.to_excel("../data/sentiment_analysis_simple.xlsx")